In [2]:
PATH = "/home/mdobrzanski/playground/IET/NLP/ustawy/"
%config Completer.use_jedi = False

In [19]:
import os
import re
import time
from collections import defaultdict, Counter, OrderedDict
from functools import reduce
import math
import pandas as pd
import requests
import collections
import torch
from transformers import BertTokenizer, BertForMaskedLM, pipeline, HerbertTokenizer

# 1

# 2

In [33]:
model_dkleczek_bert = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")

tokenizer_dkleczek_bert = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
model_Geotrend_bert = BertForMaskedLM.from_pretrained("Geotrend/bert-base-pl-cased")

tokenizer_Geotrend_bert = BertTokenizer.from_pretrained("Geotrend/bert-base-pl-cased")

In [35]:
model_Geotrend_multibert = BertForMaskedLM.from_pretrained("Geotrend/bert-base-en-fr-lt-no-pl-cased")

tokenizer_Geotrend_multibert = BertTokenizer.from_pretrained("Geotrend/bert-base-en-fr-lt-no-pl-cased")

In [38]:
nlp_dkleczek_bert = pipeline('fill-mask', model=model_dkleczek_bert, tokenizer=tokenizer_dkleczek_bert)

nlp_Geotrend_bert = pipeline('fill-mask', model=model_Geotrend_bert,
                                    tokenizer=tokenizer_Geotrend_bert)

nlp_Geotrend_multibert = pipeline('fill-mask', model=model_Geotrend_multibert,
                                   tokenizer=tokenizer_Geotrend_multibert)

In [39]:
nlps = [nlp_dkleczek_bert, nlp_Geotrend_bert, nlp_Geotrend_multibert]

In [40]:
names = ['nlp_dkleczek_bert', 'model_Geotrend_bert', 'model_Geotrend_multibert']

# 3

In [67]:
sentences = [
    'Warszawa to największe [MASK].',
    'Te zabawki należą do [MASK].',
    'Policjant przygląda się [MASK].',
    'Na środku skrzyżowania widać [MASK].',
    'Właściciel samochodu widział złodzieja z [MASK].',
    'Prezydent z premierem rozmawiali wczoraj o [MASK].',
    'Witaj drogi [MASK].'
]


In [68]:
for s in sentences:
    print(s)
    for nlp, n in zip(nlps, names):
        print(n)
        s_cp = s.replace('[MASK]', nlp.tokenizer.mask_token)
        for pred in nlp(s):
            print('\t', pred['sequence'], 'score: {}'.format(pred['score']))
        print('\n')
    print('\n')

Warszawa to największe [MASK].
nlp_dkleczek_bert
	 Warszawa to największe miasto. score: 0.955305814743042
	 Warszawa to największe województwo. score: 0.0065081301145255566
	 Warszawa to największe lotnisko. score: 0.005074873100966215
	 Warszawa to największe miasteczko. score: 0.004880122374743223
	 Warszawa to największe państwo. score: 0.00401181448251009


model_Geotrend_bert
	 Warszawa to największe miasto. score: 0.32090669870376587
	 Warszawa to największe miasta. score: 0.09014374762773514
	 Warszawa to największe woj. score: 0.07241726666688919
	 Warszawa to największe Warszawa. score: 0.04220804199576378
	 Warszawa to największe miast. score: 0.038495492190122604


model_Geotrend_multibert
	 Warszawa to największe miasto. score: 0.30215615034103394
	 Warszawa to największe miasta. score: 0.1051177904009819
	 Warszawa to największe woj. score: 0.0701567605137825
	 Warszawa to największe Warszawa. score: 0.04311792179942131
	 Warszawa to największe miast. score: 0.03967291116

# 4

In [55]:
sentences_2 = ['Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].',
               'Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK]']

In [56]:
for s in sentences_2:
    print(s)
    for nlp, n in zip(nlps, names):
        print(n)
        s_cp = s.replace('[MASK]', nlp.tokenizer.mask_token)
        for pred in nlp(s):
            print('\t', pred['sequence'], 'score: {}'.format(pred['score']))
        print('\n')
    print('\n')

Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].
nlp_dkleczek_bert
	 Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zgodził. score: 0.16777962446212769
	 Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie bał. score: 0.06955236941576004
	 Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie dowiedział. score: 0.06697978079319
	 Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie pojawił. score: 0.06324414163827896
	 Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zabił. score: 0.06266836076974869


model_Geotrend_bert
	 Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się niewiedział. score: 0.10266731679439545
	 Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie było. score: 0.0933847650885582
	 Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie stało. score: 0.04759734496474266
	 Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się ni

# 5

In [63]:
sentences_3 = ['[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.',
               'W wakacje odwiedziłem [MASK], który jest stolicą Islandii.',
               'Informatyka na [MASK] należy do najlepszych kierunków w Polsce.']

In [64]:
for s in sentences_3:
    print(s)
    for nlp, n in zip(nlps, names):
        print(n)
        s_cp = s.replace('[MASK]', nlp.tokenizer.mask_token)
        for pred in nlp(s):
            print('\t', pred['sequence'], 'score: {}'.format(pred['score']))
        print('\n')
    print('\n')

[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
nlp_dkleczek_bert
	 Woda wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza. score: 0.1671261489391327
	 Mięso wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza. score: 0.06379459798336029
	 Słońce wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza. score: 0.06294815242290497
	 Nie wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza. score: 0.058183055371046066
	 Ziemia wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza. score: 0.03854738175868988


model_Geotrend_bert
	 Jego wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza. score: 0.05196268483996391
	 Za wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza. score: 0.027973147109150887
	 Po wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza. score: 0.02120

# 6

# 7

Which of the models produced the best results?

    W większości przypadków zdecydowanie model dkleczek/bert-base-polish-cased-v1



Was any of the models able to capture Polish grammar?

    Model dkleczek/bert-base-polish-cased-v1


Was any of the models able to capture long-distant relationships between the words?

    Model dkleczek/bert-base-polish-cased-v1 - był w stanie określic płeć podmiotu, na podstawie słów wystepujących znacznie wczesniej niż modelowane słowo.


Was any of the models able to capture world knowledge?

    Model dkleczek/bert-base-polish-cased-v1 był w stanie powiedzieć że woda wrze w temp. 100 stopni (chociaż eksperymenty pokazały że to 100 nie ma tam znaczenia więc w sumie nie), natopmiast modele Geotrend zwróciały jako możliwośc poprawna stolicę Islandii (chociaż nie jako pierwszy wybór)


What are the most striking errors made by the models?

    Modele Geotrend zwracają czasem bezsensowne rezultaty, żaden z modeli nie był w stanie podac sensownego uzupełnienia dla przykładu z informatyką
